# Observaciones

+ Los indicadores se utilizarán para generar la métrica de consenso y a partir de esta generar las señales.
+ ¿Mutuar sólo parámetros del indicador o el indicador en su totalidad?
+ Probar distintas alternativas de funciones para optimizar

# Objetivos branch Prueba

* Normalizar fecha de inicio para todos los indicadores.
  + Necesito definir una fecha de inicio (está en función de la ventana de tiempo más grande)
* Disminuir el espacio de búsqueda para las ventanas de tiempo (utilizar parámetros más populares en el análisis técnico) (**DONE**)

* Disminuir el espacio de búsqueda para los parámetros reales (utilizar una partición con incrementos en unidades fijas) (**DONE**)

* Cambiar función de aptitud.

# Funciones para obtener parámetros de los indicadores y auxiliares

In [2]:
def selectNumericalReal(limits):
    '''
    Selects a real random number between the limits
    '''
    return np.random.uniform(limits[0],limits[1],1)[0]

def selectNumericalInteger(limits):
    '''
    Selects a random integer between the limits
    '''
    return np.random.choice(range(limits[0],limits[1]+1),1)[0]

def selectCategorical(kinds):
    '''
    Selects a random category
    '''
    return np.random.choice(kinds,1)[0]

#def getMaxTimeWindow(indicators):
#    '''
#    Returns the max value for the time window in all the indicators from one individual
#    '''
#    maxTimeWindow=[]
#    for indicator in indicators:
#        maxTimeWindow.append(indicator.params['timeWindow'])
#    return  np.max(maxTimeWindow)

def movingAverage(data):
    '''
    Returns simple moving average indicator
    data is a pandas series
    '''
    return np.nanmean(data)

def movingAverageExpo(lam,window,start,data):
    '''
    Returns an array with the values for an exponential moving average indicator
    data is a pandas dataframe
    '''
    MA=[]
    MA.append(data['Close'].iloc[start])
    aux_index=1
    for t in range(start-1,-1,-1):
        MA.append((1-lam)*MA[aux_index-1]+lam*data['Close'].iloc[t])
    return MA    


# Class indicator

In [3]:
class indicator:
    '''
    Creates a randomly chosen technical indicator
    '''
        
    def __init__(self):
        self.params = {}
        self.params['indicator'] = selectCategorical(indicatorKinds) 
        
        #Bollinger bands
        if self.params['indicator']=='BB':
            self.params['timeWindow'] = selectCategorical(timeWindow_bollinger_bands)
            self.params['stdDev'] = selectCategorical(sigma)
            self.params['movingAverageMethod'] ='simple' #In order to agree on how the std is calculated
            #if exponentialMA:
            #Smoothing parameter
            #self.params['priceType'] (close,low,high,open)
            
        #Moving average (simple and exponential)    
        elif self.params['indicator']=='MA':
            self.params['timeWindow'] = selectCategorical(timeWindow_moving_averages)
            self.params['movingAverageMethod'] = selectCategorical(typeMA)
            if self.params['movingAverageMethod']=='exponential':
                self.params['lambda']=selectCategorical(lambda_exp_mov_ave)
        
        #Moving averages crossover
        elif self.params['indicator']=='MAC':
            self.params['timeWindows']=[selectCategorical(timeWindow_moving_averages),selectCategorical(timeWindow_moving_averages)]
            self.params['shortTimeWindow']=min(self.params['timeWindows'])
            self.params['longTimeWindow']=max(self.params['timeWindows'])
            self.params['timeWindow']= self.params['longTimeWindow']
            self.params['movingAverageMethod'] = selectCategorical(typeMA)
            if self.params['movingAverageMethod']=='exponential':
                self.params['lambda']=np.random.uniform()
            
            
            
    def bollingerSignals(self,data,start):
        '''
        Returns a list with signals generated by a Bollinger Band indicator
        '''
        signal=[0]
        lowerBound=[]
        upperBound=[]
        window=self.params['timeWindow']-1 #Minus one in order to stay in the boundaries
        for t in range(start,-1,-1): #Upwards in time
            
            lowerBound.append(movingAverage(data['Close'].iloc[t:t+window+1]) - \
                              self.params['stdDev']*np.std(data['Close'].iloc[t:t+window+1]))
            upperBound.append(movingAverage(data['Close'].iloc[start-window+t:start + t +1]) + \
                              self.params['stdDev']*np.std(data['Close'].iloc[t:t+window+1]))
            
        auxRow=0    
        for t in range(start - 1,-1,-1):
            
            if data['Close'].iloc[t] > upperBound[auxRow+1] and data['Close'].iloc[t+1] > upperBound[auxRow]:
                signal.append(1) #Resistance created by Upper band gets broken
            elif data['Close'].iloc[t] > lowerBound[auxRow+1] and data['Close'].iloc[t+1] < lowerBound[auxRow]:
                signal.append(1) #Resistance created by Lowe band gets broken
            elif data['Close'].iloc[t] < lowerBound[auxRow+1] and data['Close'].iloc[t+1] < lowerBound[auxRow]:
                signal.append(-1) #Lower band creates a resistance
            elif data['Close'].iloc[t] < upperBound[auxRow+1] and data['Close'].iloc[t+1] < upperBound[auxRow]:
                signal.append(-1) #Support created by upperband gets broken
            else:
                signal.append(0)
            auxRow=auxRow + 1
        signal.reverse()    
        return signal #Reverse because is upwards in time
    
    def movingAverageSignals(self,data,start):
        '''
        Returns a list with signals generated by a Moving Average indicator
        '''
        signal=[0]
        MA=[]
        window=self.params['timeWindow']-1
        #Simple moving average
        if self.params['movingAverageMethod']=='simple':
            for t in range(start,-1,-1): #Upwards in time
                MA.append(movingAverage(data['Close'].iloc[t:t+window+1]))
        #Exponential moving average
        else:
            lam=self.params['lambda']
            MA=movingAverageExpo(lam,window,start,data)
             
        auxRow=0    
        for t in range(start - 1,-1,-1):
            if data['Close'].iloc[t]>MA[auxRow+1] and data['Close'].iloc[t+1]<MA[auxRow]:
                signal.append(1)
            elif data['Close'].iloc[t]<MA[auxRow+1] and data['Close'].iloc[t+1]>MA[auxRow]:
                signal.append(-1)
            else:
                signal.append(0)
        signal.reverse()        
        return signal #Reverse because is upwards in time
    
    def movingAverageCrossSignals(self,data,start):
        '''
        Returns a list with signals generated by a moving average crossover indicator
        '''
        signal=[0]
        shortMA=[]
        longMA=[]
        shortWindow=self.params['shortTimeWindow']-1
        longWindow=self.params['longTimeWindow']-1
        
        #Simple moving average
        if self.params['movingAverageMethod']=='simple':
            for t in range(start,-1,-1): #Upwards in time
                shortMA.append(movingAverage(data['Close'].iloc[t:t+shortWindow+1]))
                longMA.append(movingAverage(data['Close'].iloc[t:t+longWindow+1]))
                
        #Exponential moving average
        else:
            lam=self.params['lambda']
            shortMA=movingAverageExpo(lam,shortWindow,start,data)
            longMA=movingAverageExpo(lam,longWindow,start,data)
         
        #Calculates the difference between the MA
        #minLen=min(len(shortMA),len(longMA))
        MAC=np.asarray(shortMA)- np.asarray(longMA)
        auxRow=0    
        for t in range(start - 1,-1,-1):
            if MAC[auxRow+1]>0 and MAC[auxRow]<0:
                signal.append(1)
            elif MAC[auxRow+1]<0 and MAC[auxRow]>0:
                signal.append(-1)
            else:
                signal.append(0)
        signal.reverse()        
        return signal #Reverse because is upwards in time
    



# Class individual

In [4]:
class individual:
    '''
    creates an individual from the population
    '''
    def __init__(self):
        self.numberIndicators = selectNumericalInteger(indicatorsNumber)
        self.indicators=[]
        for i in range(0,self.numberIndicators):
            self.indicators.append(indicator())    
        #self.maxTimeWindow=getMaxTimeWindow(self.indicators)
        self.signals = pd.DataFrame(columns=range(0,self.numberIndicators +1))
        self.signals[0]=data['Date'][0:(startIndex+1)] #First column contains the dates
        self.consensus = pd.DataFrame(columns=range(0,2)) #Final signals
        self.consensus[0] = data['Date'][0:(startIndex+1)]
        self.fitness=0
        self.transactionsNumber=0
    
    def getFitness(self):
        self.fitness,self.transactionsNumber=fitness(self)
    
    def getSignals(self,data):
        '''
        Gets the trading signals for a specific indicator using the data in the dataset
        ======
        Params:
        data: A pandas data set containing the data
        ======

        ======
        Output:
        signals: Pandas dataframe with dates and signals
        ======
        '''
        #Individual signals
        for i in range(1,self.numberIndicators+1):
            indicator = self.indicators[i-1]
            if indicator.params['indicator']=='BB':
                self.signals[i] = indicator.bollingerSignals(data=data,start=startIndex)
            elif indicator.params['indicator']=='MA':
                self.signals[i] = indicator.movingAverageSignals(data=data,start=startIndex)
            elif indicator.params['indicator']=='MAC':
                self.signals[i] = indicator.movingAverageCrossSignals(data=data,start=startIndex)
                
                
        #Consensus signals
        #First row has no signals
        for row in range(1,len(self.signals)):
            if float(np.sum(self.signals.iloc[row,1:]==1))/len(self.indicators) > pConsensus:
                self.consensus.iloc[row,1]=1
            elif float(np.sum(self.signals.iloc[row,1:]==-1))/len(self.indicators) > pConsensus:
                self.consensus.iloc[row,1]=-1
            else:
                self.consensus.iloc[row,1]=0
                
        self.consensus.iloc[0,1]=0
        self.consensus.columns = ['Date','Signal']


# Fitness

In [5]:
def fitness(individual):
    '''
    Returns the fitness of an individual
    using the consensus trading signals.
    
    Based on excess return from buy-and-hold strategy
    
    '''
    #Filters the data by date
    filteredData=data.iloc[0:(startIndex+1),]
    
    #flag first buy
    flagFirstBuy=True
    #Current Market Position (in/out)
    marketPos="out"
    #continous return from the buy and sell movements
    ret=0
    #last buy price
    lastBuyPrice=0
    #Transactions count
    transactionsCount=0
    
    for t in range(startIndex-1,0,-1): #Not including zero since we are using closing prices (trading next day after the signal)
        #First buy signal
        if individual.consensus['Signal'].iloc[t]==1 and flagFirstBuy==True:
            marketPos="in"
            lastBuyPrice=filteredData['Close'].iloc[t]
            transactionsCount=transactionsCount+1
            flagFirstBuy=False
        #Rest of buy trading signals    
        elif individual.consensus['Signal'].iloc[t]==1 and flagFirstBuy==False and marketPos=="out":
            marketPos="in"
            lastBuyPrice=filteredData['Close'].iloc[t]
            transactionsCount=transactionsCount+1
        #Sell trading signals (NO SHORT SALES)
        elif individual.consensus['Signal'].iloc[t]==-1 and flagFirstBuy==False and marketPos=="in":
            marketPos="out"
            ret=ret + np.log(filteredData['Close'].iloc[t]/lastBuyPrice)
            transactionsCount=transactionsCount+1
    
    #Forces to sell in the last trading day if market position is in the market
    if marketPos=="in":
        ret=ret + np.log(filteredData['Close'].iloc[0]/lastBuyPrice)
        transactionsCount=transactionsCount+1
    
    #Adds all the transaction costs
    ret=ret + transactionsCount*np.log((1-transactionCost)/(1+transactionCost))
    
    #Excess return
    excess=ret - buyHold
    
    #Using roulette selection, so excess should be positive
    #Need to investigate another selection methods
    if excess<=0 or transactionsCount==0:
        excess = np.min([buyHold,0.001])
    
    return excess,transactionsCount


# Cruza

In [6]:
def crossover(population,probabilities):
    #Selects the parents according to their fitness
    parents=np.random.choice(population,size=2,replace=False,p=probabilities)

    #Selects crossover point (avoids ends)
    crossoverPoint=np.random.choice(range(1,len(parents[0].indicators)-1))

    #Creates the children
    child1=parents[0]
    child2=parents[1]
    child1.indicators=parents[0].indicators[0:crossoverPoint] + parents[1].indicators[crossoverPoint:]
    child2.indicators=parents[1].indicators[0:crossoverPoint] + parents[0].indicators[crossoverPoint:]
    
    return [child1,child2]


# Carga de librerías e inicialización de parámetros

In [8]:
import pandas as pd
import numpy as np
import copy as cp
#import pdb
import matplotlib.pyplot as plt
%matplotlib inline
import time


####################################### FOR THE MOMENT GLOBAL VARIABLES ###################################
#Prepares the data
#Data from google finance
data=pd.read_csv('amxl.csv')

#The next line was neccesary for handling Date column, some hidden characters appeared when reading the csv file
data.columns = ['Date','Open','High','Low','Close','Volume'] 

#Converts string to dates
data['Date'] = pd.to_datetime(data['Date'])

#Sorts the data from oldest to newest
#data=data.sort_values(by='Date')

#Need to be careful with how much data I use since this makes the algorithm slow
#data=data.iloc[-365*1:]

#Defines the domain for the parameters
#The format is:
#[lowerBound,upperBound] For numerical parameters
#[category1,...,categoryN] For categorical parameters
sigma=[0.5,1,1.5,2,2.5,3]
timeWindow_bollinger_bands=[20,30,40,50]
timeWindow_moving_averages=[5,10,50,100,200]
lambda_exp_mov_ave=[0.1,0.5,0.7,0.8,0.9]
typeMA=['simple','exponential']
indicatorKinds=['BB','MA','MAC']
indicatorsNumber=[5,5]
startDate=pd.to_datetime("2016-11-14")
startIndex=data[data.Date==startDate].index[0]

#Defines the proportion for minimum consensus
pConsensus=0.5

#Defines the per-transaction cost
transactionCost=0.25/100

#Buy and hold return
firstPrice=data['Close'].iloc[startIndex-1] #Same point time that we have a trading signal
lastPrice=data['Close'].iloc[1] #Using 1 in order to coincide with fitness function
buyHold=np.log(1+(lastPrice/firstPrice)*((1-transactionCost)/(1+transactionCost)))

#Defines the mutation probability
pMutation=0.1

populationSize=10
population=[]
newPopulation=[]
numberGenerations=20
countGeneration=1

currentBestInd=[]
currentBestGain=0
bestInd=[]
bestGain=0

##############################################################################################################

        

# Iteraciones para el proceso de evolución

In [9]:
#Creates initial population
for i in range(0,populationSize):
    population.append(individual())
    
startTime=time.ctime()   
while countGeneration<=numberGenerations:
    
    #Gets population signals
    for i in range(0,len(population)):
        population[i].getSignals(data=data)    

    #Gets individual's fitness
    probabilities=[]
    for entity in population:
        entity.getFitness()
        probabilities.append(entity.fitness)
        
    #Stores the best individual for the current generation
    currentBestInd=population[np.argmax(probabilities)]
    currentBestGain=currentBestInd.fitness
    
    #Stores the best individual among all generations
    if currentBestGain > bestGain:
        bestInd = cp.deepcopy(currentBestInd)
        bestGain = currentBestGain
        
    
    print 'Max gain so far: ' + str(bestGain) + ' Generation: ' + str(countGeneration) \
     + ' Number of transactions for best individual: ' + str(bestInd.transactionsNumber) + \
        " Buy-Hold: " + str(buyHold)
    
    #normalizes in order to get selection probabilities
    probabilities=probabilities/np.sum(probabilities)
    
    #Creates the new population
    
    #Elitism (keeping the best individual so far)
    newPopulation.append(bestInd)
    
    #Crossover
    while len(newPopulation)<len(population):
        
        #Creates children
        children=crossover(population,probabilities)
        newPopulation.append(children[0])
        newPopulation.append(children[1])
    
    #Mutates the population excluding the best individual
    for i in range(1,len(newPopulation)):
        u=np.random.uniform()
        if u<pMutation:
            newPopulation[i]=individual()
    
    #Replace the old population
    population=cp.deepcopy(newPopulation)
    newPopulation=[]
    countGeneration=countGeneration + 1
finishTime=time.ctime()

print 'Start: ' + startTime +' Finish: ' + finishTime


Max gain so far: 0.001 Generation: 1 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 2 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 3 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 4 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 5 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 6 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 7 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 8 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Generation: 9 Number of transactions for best individual: 2 Buy-Hold: 0.917213148911
Max gain so far: 0.001 Gener

# To Do
* Implement more indicators
* Tournament based selection

